In [5]:
import pandas as pd
bad = pd.read_csv('../data/bad/bad.csv')
normal=pd.read_csv('../data/neautral/neutral.csv')
hiniku_=pd.read_csv('../data/hiniku/hiniku_with_word.csv')

In [6]:
positivelist = list(hiniku_.dropna().positiveword.values)
hinikulist = list(hiniku_.dropna().hinikuword.values)
badwordlist = list(bad.word)#悪口単語をリストで抽出

In [107]:
'''
悪口ワードの含有
皮肉ワードの含有
ポジティブワードの含有

[形態素解析した時の単語数],[単純に悪口単語が文章に含まれているのかのフラグ], [形態素ごとに現れる悪口数]
[皮肉単語が含まれているかのフラグ],[形態素ごとに現れる皮肉数]
[ポジティブな単語が含まれているかのフラグ], [形態素ごとに現れるポジティブ単語数]

'''
import pandas as pd
# df = bad.copy()
df = pd.concat([hiniku_, normal])
df = pd.concat([bad, df])
import sys
sys.path.append('/Users/nagataeiki/.pyenv/versions/3.7.4/lib/python3.7/site-packages')
import numpy as np
from janome.tokenizer import Tokenizer
from tqdm import tqdm

tokenizer = Tokenizer()
#悪口が含まれている数のカウント, 単に含まれているのかのフラグを返す
def get_word_count(text, wordlist):
    textlist = []
    count=0
    flg=0
    for t in tokenizer.tokenize(text):
        if t.surface in wordlist:
            count+=1
    for w in wordlist:
        if w in text:
            flg=1
            return count, flg
    
    return count,flg


#dfと悪口辞書を入れて新しい特徴量を含んだdfを返す
def make_bad_feature(df, badwordlist, positivelist, hinikulist):
    col=["content","mophologics_num", "bad_per_mophologic", "simple_bad_flg","target", "hiniku_per_mophologic",\
         "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
    basedf = pd.DataFrame(columns=col)
    tokenizer = Tokenizer()
    for r in tqdm(df.iterrows()):
        text = str(r[1].content)
        target = r[1].type
        num = len(tokenizer.tokenize(text))
        badcount,badflg = get_word_count(text, badwordlist)
        hinikucount, hinikuflg = get_word_count(text, hinikulist)
        positivecount, positiveflg = get_word_count(text, positivelist)
        newdf = pd.DataFrame(np.array([text, num, badcount,badflg,target,\
                                       hinikucount, hinikuflg, positivecount,\
                                       positiveflg]).reshape(1,-1),columns=col)
        basedf = pd.concat([basedf, newdf])

    basedf["target"] = basedf.apply(lambda x: int(x["target"]), axis=1)
    return basedf
df =make_bad_feature(df, badwordlist, positivelist, hinikulist)
df.head(2)
print(df.shape)


750it [00:06, 109.38it/s]

(750, 9)


In [14]:
# df.head()

In [12]:
# hiniku_ = pd.read_csv('../data/hiniku/hiniku_with_word.csv')

# col=["content","mophologics_num", "bad_word_num", "simple_bad_flg","target", "hinikuword", "positiveword"]


In [22]:
# '''
# 1対1で予測をする時
# '''

# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import f1_score
# x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
# svm = SVC(C=1.0, random_state=42)
# svm.fit(x_train, y_train)

# pred_train = svm.predict(x_train)
# pred_test = svm.predict(x_test)
# print(f1_score(y_train, pred_train))
# print(f1_score(y_test, pred_test))

In [97]:
"""
1対その他での学習
"""


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import f1_score

def train_and_predict(df):
    col=["mophologics_num", "bad_per_mophologic", "simple_bad_flg", "hiniku_per_mophologic",\
             "simple_hiniku_flg", "positive_per_mophologic", "simple_positive_flg"]
    X = df[col]
    y = df.target
    x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    svm = SVC(C=1.0, random_state=42)
    model = OneVsRestClassifier(svm)
    model.fit(x_train, y_train)

    return {
        "y_train":y_train,
        "y_test":y_test,
        "pred_train":  model.predict(x_train),
        "pred_test": model.predict(x_test)
    }, x_train,x_test
    # print(f1_score(y_train, pred_train))
    # print(f1_score(y_test, pred_test))

In [25]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

'''
mxm,ここでは3x3のマトリックスを作成する
そこからf1値を計算する（同ディレクトリの画像参照）
'''

def create_3x3_matrix(true_y, pred_y):
    matrix = np.zeros(9)

    for t,p in zip(true_y, pred_y):
        if t==0:
            if p==0:matrix[0]+=1
            if p==1:matrix[1]+=1
            if p==2:matrix[2]+=1
        elif t==1:
            if p==0:matrix[3]+=1
            if p==1:matrix[4]+=1
            if p==2:matrix[5]+=1
        elif t==2:
            if p==0:matrix[6]+=1
            if p==1:matrix[7]+=1
            if p==2:matrix[8]+=1

    return matrix.reshape(3,3)
    
    


In [26]:
matrix=create_3x3_matrix(y_train, pred_train)
matrix

array([[202.,   5.,   0.],
       [  3., 189.,   6.],
       [  0.,  10., 185.]])

In [27]:
def calc_scores(matrix):
    Recalls = np.zeros(3)
    for i in range(3):
        Recalls[i] = matrix[i][i]/sum(matrix[i])
    Precisions = np.zeros(3)
    pred_sum = matrix.sum(axis=0)
    for i, ps in enumerate(pred_sum):
        Precisions[i] = matrix[i][i]/ps
    f1_score = 2*Recalls.mean()*Precisions.mean()/(Recalls.mean()+Precisions.mean())
    return {
        "recalls":Recalls,
        "Precisions":Precisions,
        "f1":f1_score
    }

In [28]:
calc_scores(matrix)

{'recalls': array([0.97584541, 0.95454545, 0.94871795]),
 'Precisions': array([0.98536585, 0.92647059, 0.96858639]),
 'f1': 0.9599218905720232}

In [124]:
# Recalls = np.zeros(3)
# for i in range(3):
#     Recalls[i] = matrix[i][i]/sum(matrix[i])
# Recalls

array([0.75362319, 0.93434343, 0.37948718])

In [133]:
# Precisions = np.zeros(3)
# pred_sum = matrix.sum(axis=0)
# for i, ps in enumerate(pred_sum):
#     Precisions[i] = matrix[i][i]/ps
# Precisions

array([0.59541985, 0.87677725, 0.58267717])

In [142]:
# f1_score = 2*Recalls.mean()*Precisions.mean()/(Recalls.mean()+Precisions.mean())
# f1_score

0.6870482798110917

(2352, 4)

In [72]:
def print_miss_pred(true_y, pred):
    arr = []
    pred_judge=[]
    true_judge=[]
    for i in range(len(pred)):
        if pred[i]!=true_y.values[i]:
#             print(pred[i], true_y.values[i])
            arr.append(i)
            pred_judge.append(pred[i])
            true_judge.append(true_y.values[i])

    def print_str_label(num):
        if num==0:
            return "ニュートラル"
        if num==1:
            return "悪口"
        if num==2:
            return "皮肉"
        return "バグあり"

    for i,idx in enumerate(arr):
        display(df.iloc[idx].content)
        print('true label  *',true_judge[i] , (print_str_label(true_judge[i])))
        print('pred label  *',pred_judge[i], (print_str_label(pred_judge[i])))
        print("------------------------------------------------------------------------------------")
        
    return pred_judge, true_judge

In [73]:
print_miss_pred(y_test, pred_test)

'人間のクズが'

true label  * 1 悪口
pred label  * 2 皮肉
------------------------------------------------------------------------------------


'下手糞な編集すんなや・・・'

true label  * 2 皮肉
pred label  * 1 悪口
------------------------------------------------------------------------------------


'ごっみ'

...

'ヤリマンもんじゃ店'

true label  * 1 悪口
pred label  * 0 ニュートラル
------------------------------------------------------------------------------------


'無資格でしょ'

true label  * 2 皮肉
pred label  * 1 悪口
------------------------------------------------------------------------------------


([2, 1, 1, 0, 1], [1, 2, 2, 1, 2])

In [66]:
p,t = print_miss_pred(y_test, pred_test)

2 1
1 2
1 2
0 1
1 2


'人間のクズが'

------------------------------------------------------------------------------------


'下手糞な編集すんなや・・・'

------------------------------------------------------------------------------------


...

------------------------------------------------------------------------------------


'ヤリマンもんじゃ店'

------------------------------------------------------------------------------------


'無資格でしょ'

------------------------------------------------------------------------------------


In [68]:
p

[2, 1, 1, 0, 1]

In [69]:
t

[0, 2, 1, 2, 0]

In [98]:
df1=pd.read_csv('../data/clean/bad.csv')
df2=pd.read_csv('../data/clean/hiniku.csv')
df3=pd.read_csv('../data/clean/normal.csv')
df1 = pd.concat([df1, df2])
df1 = pd.concat([df1, df3])
print(df1.shape)
df1.head(3)

(2352, 4)


,Unnamed: 0,content,type,word
0,0,ガイジ,1,ガイジ
1,1,くどい,1,くどい
2,2,偽物 ただの釣り動画,1,偽物


In [106]:
# df1 = make_bad_feature(df1, badwordlist=badwordlist, positivelist=positivelist, hinikulist=hinikulist)
# df1.to_csv('../data/clean/mount_data.csv', index=False)
df1.to_excel('../data/clean/mount_data.xlsx', index=False)

In [108]:
pred,a,b = train_and_predict(df)
pred.keys()

dict_keys(['y_train', 'y_test', 'pred_train', 'pred_test'])

In [109]:
train_matrix = create_3x3_matrix(pred["y_train"], pred["pred_train"])
test_matrix  = create_3x3_matrix(pred['y_test'],  pred['pred_test'])

In [110]:
calc_scores(train_matrix)

{'recalls': array([0.71014493, 0.93939394, 0.57948718]),
 'Precisions': array([0.68372093, 0.88995215, 0.64204545]),
 'f1': 0.740784123632467}

In [111]:
calc_scores(test_matrix)

{'recalls': array([0.62790698, 0.92307692, 0.65454545]),
 'Precisions': array([0.62790698, 0.88888889, 0.67924528]),
 'f1': 0.7335916749755873}

In [112]:
print_miss_pred(pred['y_test'],  pred['pred_test'])

'ガイジ'

true label  * 0 ニュートラル
pred label  * 2 皮肉
------------------------------------------------------------------------------------


'糞みたいな店員だな'

true label  * 0 ニュートラル
pred label  * 2 皮肉
------------------------------------------------------------------------------------


'人間のクズが'

...

'掛け声うるさい！'

true label  * 0 ニュートラル
pred label  * 2 皮肉
------------------------------------------------------------------------------------


'トリ頭だから言ってんだろ'

true label  * 1 悪口
pred label  * 2 皮肉
------------------------------------------------------------------------------------


([2,
  2,
  2,
  2,
  0,
  2,
  0,
  2,
  0,
  0,
  1,
  0,
  0,
  2,
  2,
  0,
  1,
  0,
  2,
  0,
  0,
  0,
  2,
  0,
  2,
  2,
  1,
  0,
  2,
  1,
  0,
  2,
  0,
  2,
  0,
  1,
  1,
  2,
  2],
 [0,
  0,
  1,
  0,
  2,
  0,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  1,
  0,
  2,
  0,
  2,
  0,
  2,
  2,
  2,
  0,
  2,
  0,
  0,
  2,
  2,
  0,
  2,
  2,
  0,
  1,
  0,
  2,
  2,
  0,
  0,
  1])